In [ ]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas as pd
import seaborn as sns
from flask import Flask, request
import matplotlib.pyplot as plt

In [ ]:
con = psycopg2.connect(host='localhost', database='imobiliaria',user='postgres', password='postgres')
cur = con.cursor()

app = Flask(__name__)

In [ ]:
# Criar endpoint de get, que deve selecionar os dados dentro de um intervalo de area, 
# plotar o gráfico de dispersão em que o eixo x é a área e o eixo y o preco e retornar o json com esses dados;


@app.route("/area/<a>/<b>", methods =['GET'])
def area(a,b):
    query = "SELECT area, preco FROM info_casas WHERE  area > {a}  AND area < {b}".format(a=a,b=b)
    try:
        df = pd.read_sql(query,con)
        sns.scatterplot(x=df.area, y=df.preco)
        plt.show()
        return df.to_json(orient='records'), 200
    except:
        return "Não foi possível fazer SELECT na tabela casas.", 500


In [ ]:
#Criar endpoint de get, que deve selecionar os 15 dados com maior preco em ordem decrescente, 
#plotar o gráfico de linha em que o eixo x é o preco e o eixo y a qte_quartos e retornar o json com esses dados;


@app.route("/15", methods =['GET'])
def top():
    
    query = "SELECT qte_quartos, preco FROM info_casas ORDER BY preco DESC limit 15"
    try:
        df = pd.read_sql(query,con)
        sns.lineplot(x=df.preco, y=df.qte_quartos)
        plt.show()
        return df.to_json(orient='records'), 200
    except:
        return "Não foi possível fazer SELECT na tabela casas.", 500


In [ ]:
#Criar endpoint de get, que deve selecionar todos os dados com determinada quantidade de qte_banheiros,
#plotar o gráfico de barra em que o eixo x é o qte_banheiros e o eixo y a qte_quartos e retornar o json com esses dados;



@app.route("/ban/<a>/<b>", methods =['GET'])
def ban(a,b):
    query = "SELECT qte_banheiros, qte_quartos FROM info_casas WHERE qte_banheiros > {a}  AND qte_banheiros < {b}".format(a=a,b=b)
    try:
        df = pd.read_sql(query,con)
        sns.countplot(x=df.qte_banheiros, data=df)
        plt.show()
        return df.to_json(orient='records'), 200
    except:
        return "Não foi possível fazer SELECT na tabela casas.", 500


In [ ]:
#Criar endpoint de put, que atualize os dados pelo cep;


@app.route("/atualizacao", methods =['PUT'])
def atualizacao():
    querystring_args = request.args.to_dict()
    post_args = request.json
    try:
        sql ="UPDATE info_casas SET qte_quartos = {}, qte_banheiros = {}, area={}, preco={} WHERE cep = {}".format(post_args["qte_quartos"] ,post_args["qte_banheiros"], post_args["area"], post_args["preco"], post_args["cep"])
        cur.execute(sql)
        con.commit()
        return "UPDATE com sucesso", 200
    except:
        return "Não foi possível fazer UPDATE na tabela casas.", 500


In [ ]:
# Criar endpoint de delete, que delete os dados pelo cep;



@app.route("/delete", methods =['DELETE'])
def delete():
    querystring_args = request.args.to_dict()
    post_args = request.json
    try:
        sql ="DELETE FROM info_casas  WHERE cep = {}".format(post_args["cep"])
        cur.execute(sql)
        con.commit()
        return "Delete com sucesso", 200
    except:
        return "Não foi possível fazer o DELETE na tabela casas.", 500


In [ ]:
def imobiliaria(request):
    if request.path == '/area/<a>/<b>':
        return area()
    elif request.path == "/15":
        return top()
    elif request.path == "/ban/<a>/<b>":
        return ban()
    elif request.path == "/atualizacao":
        return atualizacao()
    elif request.path == "/delete":
        return delete()
    else:
        return 'Método não suportado.', 400


app.run()